### Importando as Bibliotecas e Funções

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

def drop_reset_index(df):
    df = df.dropna()
    df = df.reset_index(drop=True)
    df.index += 1
    return df

pd.set_option('display.max_columns', None)

### Importando a Base de Dados

In [ ]:
base = pd.read_csv("https://github.com/futpythontrader/Live_FutPythonTrader/raw/refs/heads/main/Base_FutPythonTrader.csv")
df = base[['League','Season','Date','Home','Away',
           'Odd_H_FT','Odd_D_FT','Odd_A_FT',
           'Odd_1X','Odd_12','Odd_X2',
           'Goals_H_FT','Goals_A_FT']].copy()
df = drop_reset_index(df)
display(df)

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df['League'].unique()

### Criando Variáveis Secundárias

In [ ]:
# Probabilidades
df['p_H'] = 1/df['Odd_H_FT']
df['p_D'] = 1/df['Odd_D_FT']
df['p_A'] = 1/df['Odd_A_FT']

# Coeficiente de Variação das Probabilidades
desvio_padrao = df[['p_H', 'p_D', 'p_A']].std(axis=1)
media = df[['p_H', 'p_D', 'p_A']].mean(axis=1)
CV_Prob = desvio_padrao / media
df['CV_Prob'] = CV_Prob

n_per = 5

# Média dos Pontos (PPG)
df['Ptos_H'] = np.where(df['Goals_H_FT'] > df['Goals_A_FT'], 3,
               np.where(df['Goals_H_FT'] == df['Goals_A_FT'], 1, 0))
df['Ptos_A'] = np.where(df['Goals_H_FT'] < df['Goals_A_FT'], 3,
               np.where(df['Goals_H_FT'] == df['Goals_A_FT'], 1, 0))
df['Media_Ptos_H'] = df.groupby('Home')['Ptos_H'].rolling(window=n_per, min_periods=n_per).mean().reset_index(0, drop=True)
df['Media_Ptos_H'] = df.groupby('Home')['Media_Ptos_H'].shift(1)
df['Media_Ptos_A'] = df.groupby('Away')['Ptos_A'].rolling(window=n_per, min_periods=n_per).mean().reset_index(0, drop=True)
df['Media_Ptos_A'] = df.groupby('Away')['Media_Ptos_A'].shift(1)
df['DesvPad_Ptos_H'] = df.groupby('Home')['Ptos_H'].rolling(window=n_per, min_periods=n_per).std().reset_index(0, drop=True)
df['DesvPad_Ptos_H'] = df.groupby('Home')['DesvPad_Ptos_H'].shift(1)
df['DesvPad_Ptos_A'] = df.groupby('Away')['Ptos_A'].rolling(window=n_per, min_periods=n_per).std().reset_index(0, drop=True)
df['DesvPad_Ptos_A'] = df.groupby('Away')['DesvPad_Ptos_A'].shift(1)
df['CV_Ptos_H'] = df['DesvPad_Ptos_H'] / df['Media_Ptos_H']
df['CV_Ptos_A'] = df['DesvPad_Ptos_A'] / df['Media_Ptos_A']

# Média Gols Marcados
df['Media_GM_H'] = df.groupby('Home')['Goals_H_FT'].rolling(window=n_per, min_periods=n_per).mean().reset_index(0,drop=True)
df['Media_GM_A'] = df.groupby('Away')['Goals_A_FT'].rolling(window=n_per, min_periods=n_per).mean().reset_index(0,drop=True)
df['Media_GM_H'] = df.groupby('Home')['Media_GM_H'].shift(1)
df['Media_GM_A'] = df.groupby('Away')['Media_GM_A'].shift(1)
df['DesvPad_GM_H'] = df.groupby('Home')['Goals_H_FT'].rolling(window=n_per, min_periods=n_per).std().reset_index(0,drop=True)
df['DesvPad_GM_A'] = df.groupby('Away')['Goals_A_FT'].rolling(window=n_per, min_periods=n_per).std().reset_index(0,drop=True)
df['DesvPad_GM_H'] = df.groupby('Home')['DesvPad_GM_H'].shift(1)
df['DesvPad_GM_A'] = df.groupby('Away')['DesvPad_GM_A'].shift(1)
df['CV_GM_H'] = df['DesvPad_GM_H'] / df['Media_GM_H']
df['CV_GM_A'] = df['DesvPad_GM_A'] / df['Media_GM_A']

# Média de Gols Sofridos
df['Media_GS_H'] = df.groupby('Home')['Goals_A_FT'].rolling(window=n_per, min_periods=n_per).mean().reset_index(0,drop=True)
df['Media_GS_A'] = df.groupby('Away')['Goals_H_FT'].rolling(window=n_per, min_periods=n_per).mean().reset_index(0,drop=True)
df['Media_GS_H'] = df.groupby('Home')['Media_GS_H'].shift(1)
df['Media_GS_A'] = df.groupby('Away')['Media_GS_A'].shift(1)
df['DesvPad_GS_H'] = df.groupby('Home')['Goals_A_FT'].rolling(window=n_per, min_periods=n_per).std().reset_index(0,drop=True)
df['DesvPad_GS_A'] = df.groupby('Away')['Goals_H_FT'].rolling(window=n_per, min_periods=n_per).std().reset_index(0,drop=True)
df['DesvPad_GS_H'] = df.groupby('Home')['DesvPad_GS_H'].shift(1)
df['DesvPad_GS_A'] = df.groupby('Away')['DesvPad_GS_A'].shift(1)
df['CV_GS_H'] = df['DesvPad_GS_H'] / df['Media_GS_H']
df['CV_GS_A'] = df['DesvPad_GS_A'] / df['Media_GS_A']

# Média de Saldo de Gols
df['SG_H'] = df['Goals_H_FT'] - df['Goals_A_FT']
df['SG_A'] = df['Goals_A_FT'] - df['Goals_H_FT']
df['Media_SG_H'] = df.groupby('Home')['SG_H'].rolling(window=n_per, min_periods=n_per).mean().reset_index(0,drop=True)
df['Media_SG_A'] = df.groupby('Away')['SG_A'].rolling(window=n_per, min_periods=n_per).mean().reset_index(0,drop=True)
df['Media_SG_H'] = df.groupby('Home')['Media_SG_H'].shift(1)
df['Media_SG_A'] = df.groupby('Away')['Media_SG_A'].shift(1)
df['DesvPad_SG_H'] = df.groupby('Home')['SG_H'].rolling(window=n_per, min_periods=n_per).std().reset_index(0,drop=True)
df['DesvPad_SG_A'] = df.groupby('Away')['SG_A'].rolling(window=n_per, min_periods=n_per).std().reset_index(0,drop=True)
df['DesvPad_SG_H'] = df.groupby('Home')['DesvPad_SG_H'].shift(1)
df['DesvPad_SG_A'] = df.groupby('Away')['DesvPad_SG_A'].shift(1)
df['CV_SG_H'] = df['DesvPad_SG_H'] / df['Media_SG_H']
df['CV_SG_A'] = df['DesvPad_SG_A'] / df['Media_SG_A']

# Média de Saldo de Gols Ponderado pela Probabilidade do Time
df['SG_H_01'] = (df['Goals_H_FT'] - df['Goals_A_FT']) / df['p_H']
df['SG_A_01'] = (df['Goals_A_FT'] - df['Goals_H_FT']) / df['p_A']
df['Media_SG_H_01'] = df.groupby('Home')['SG_H_01'].rolling(window=n_per, min_periods=n_per).mean().reset_index(0,drop=True)
df['Media_SG_A_01'] = df.groupby('Away')['SG_A_01'].rolling(window=n_per, min_periods=n_per).mean().reset_index(0,drop=True)
df['Media_SG_H_01'] = df.groupby('Home')['Media_SG_H_01'].shift(1)
df['Media_SG_A_01'] = df.groupby('Away')['Media_SG_A_01'].shift(1)
df['DesvPad_SG_H_01'] = df.groupby('Home')['SG_H_01'].rolling(window=n_per, min_periods=n_per).std().reset_index(0,drop=True)
df['DesvPad_SG_A_01'] = df.groupby('Away')['SG_A_01'].rolling(window=n_per, min_periods=n_per).std().reset_index(0,drop=True)
df['DesvPad_SG_H_01'] = df.groupby('Home')['DesvPad_SG_H_01'].shift(1)
df['DesvPad_SG_A_01'] = df.groupby('Away')['DesvPad_SG_A_01'].shift(1)
df['CV_SG_H_01'] = df['DesvPad_SG_H_01'] / df['Media_SG_H_01']
df['CV_SG_A_01'] = df['DesvPad_SG_A_01'] / df['Media_SG_A_01']

# Média de Saldo de Gols Ponderado pela Probabilidade do Adversário
df['SG_H_02'] = (df['Goals_H_FT'] - df['Goals_A_FT']) / df['p_A']
df['SG_A_02'] = (df['Goals_A_FT'] - df['Goals_H_FT']) / df['p_H']
df['Media_SG_H_02'] = df.groupby('Home')['SG_H_02'].rolling(window=n_per, min_periods=n_per).mean().reset_index(0,drop=True)
df['Media_SG_A_02'] = df.groupby('Away')['SG_A_02'].rolling(window=n_per, min_periods=n_per).mean().reset_index(0,drop=True)
df['Media_SG_H_02'] = df.groupby('Home')['Media_SG_H_02'].shift(1)
df['Media_SG_A_02'] = df.groupby('Away')['Media_SG_A_02'].shift(1)
df['DesvPad_SG_H_02'] = df.groupby('Home')['SG_H_02'].rolling(window=n_per, min_periods=n_per).std().reset_index(0,drop=True)
df['DesvPad_SG_A_02'] = df.groupby('Away')['SG_A_02'].rolling(window=n_per, min_periods=n_per).std().reset_index(0,drop=True)
df['DesvPad_SG_H_02'] = df.groupby('Home')['DesvPad_SG_H_02'].shift(1)
df['DesvPad_SG_A_02'] = df.groupby('Away')['DesvPad_SG_A_02'].shift(1)
df['CV_SG_H_02'] = df['DesvPad_SG_H_02'] / df['Media_SG_H_02']
df['CV_SG_A_02'] = df['DesvPad_SG_A_02'] / df['Media_SG_A_02']

# Média do Valor do Gol
df['VG_H'] = df['Goals_H_FT'] * df['p_A']
df['VG_A'] = df['Goals_A_FT'] * df['p_H']
df['Media_VG_H'] = df.groupby('Home')['VG_H'].rolling(window=n_per, min_periods=n_per).mean().reset_index(0,drop=True)
df['Media_VG_A'] = df.groupby('Away')['VG_A'].rolling(window=n_per, min_periods=n_per).mean().reset_index(0,drop=True)
df['Media_VG_H'] = df.groupby('Home')['Media_VG_H'].shift(1)
df['Media_VG_A'] = df.groupby('Away')['Media_VG_A'].shift(1)
df['DesvPad_VG_H'] = df.groupby('Home')['VG_H'].rolling(window=n_per, min_periods=n_per).std().reset_index(0,drop=True)
df['DesvPad_VG_A'] = df.groupby('Away')['VG_A'].rolling(window=n_per, min_periods=n_per).std().reset_index(0,drop=True)
df['DesvPad_VG_H'] = df.groupby('Home')['DesvPad_VG_H'].shift(1)
df['DesvPad_VG_A'] = df.groupby('Away')['DesvPad_VG_A'].shift(1)
df['CV_VG_H'] = df['DesvPad_VG_H'] / df['Media_VG_H']
df['CV_VG_A'] = df['DesvPad_VG_A'] / df['Media_VG_A']

# Custo do Gol Original
df['CG_H_00'] = df['p_H'] / df['Goals_H_FT']
df['CG_A_00'] = df['p_A'] / df['Goals_A_FT']
df.replace(np.inf, 1, inplace=True)
df['Media_CG_H_00'] = df.groupby('Home')['CG_H_00'].rolling(window=n_per, min_periods=n_per).mean().reset_index(0,drop=True)
df['Media_CG_A_00'] = df.groupby('Away')['CG_A_00'].rolling(window=n_per, min_periods=n_per).mean().reset_index(0,drop=True)
df['Media_CG_H_00'] = df.groupby('Home')['Media_CG_H_00'].shift(1)
df['Media_CG_A_00'] = df.groupby('Away')['Media_CG_A_00'].shift(1)
df['DesvPad_CG_H_00'] = df.groupby('Home')['CG_H_00'].rolling(window=n_per, min_periods=n_per).std().reset_index(0,drop=True)
df['DesvPad_CG_A_00'] = df.groupby('Away')['CG_A_00'].rolling(window=n_per, min_periods=n_per).std().reset_index(0,drop=True)
df['DesvPad_CG_H_00'] = df.groupby('Home')['DesvPad_CG_H_00'].shift(1)
df['DesvPad_CG_A_00'] = df.groupby('Away')['DesvPad_CG_A_00'].shift(1)
df['CV_CG_H_00'] = df['DesvPad_CG_H_00'] / df['Media_CG_H_00']
df['CV_CG_A_00'] = df['DesvPad_CG_A_00'] / df['Media_CG_A_00']

# Custo do Gol 1.0
df['CG_H_01'] = df['Goals_H_FT'] / df['p_H']
df['CG_A_01'] = df['Goals_A_FT'] / df['p_A']
df['Media_CG_H_01'] = df.groupby('Home')['CG_H_01'].rolling(window=n_per, min_periods=n_per).mean().reset_index(0,drop=True)
df['Media_CG_A_01'] = df.groupby('Away')['CG_A_01'].rolling(window=n_per, min_periods=n_per).mean().reset_index(0,drop=True)
df['Media_CG_H_01'] = df.groupby('Home')['Media_CG_H_01'].shift(1)
df['Media_CG_A_01'] = df.groupby('Away')['Media_CG_A_01'].shift(1)
df['DesvPad_CG_H_01'] = df.groupby('Home')['CG_H_01'].rolling(window=n_per, min_periods=n_per).std().reset_index(0,drop=True)
df['DesvPad_CG_A_01'] = df.groupby('Away')['CG_A_01'].rolling(window=n_per, min_periods=n_per).std().reset_index(0,drop=True)
df['DesvPad_CG_H_01'] = df.groupby('Home')['DesvPad_CG_H_01'].shift(1)
df['DesvPad_CG_A_01'] = df.groupby('Away')['DesvPad_CG_A_01'].shift(1)
df['CV_CG_H_01'] = df['DesvPad_CG_H_01'] / df['Media_CG_H_01']
df['CV_CG_A_01'] = df['DesvPad_CG_A_01'] / df['Media_CG_A_01']

# Custo do Gol 2.0
df['CG_H_02'] = (df['Goals_H_FT'] / 2) + (df['p_H'] / 2)
df['CG_A_02'] = (df['Goals_A_FT'] / 2) + (df['p_A'] / 2)
df['Media_CG_H_02'] = df.groupby('Home')['CG_H_02'].rolling(window=n_per, min_periods=n_per).mean().reset_index(0,drop=True)
df['Media_CG_A_02'] = df.groupby('Away')['CG_A_02'].rolling(window=n_per, min_periods=n_per).mean().reset_index(0,drop=True)
df['Media_CG_H_02'] = df.groupby('Home')['Media_CG_H_02'].shift(1)
df['Media_CG_A_02'] = df.groupby('Away')['Media_CG_A_02'].shift(1)
df['DesvPad_CG_H_02'] = df.groupby('Home')['CG_H_02'].rolling(window=n_per, min_periods=n_per).std().reset_index(0,drop=True)
df['DesvPad_CG_A_02'] = df.groupby('Away')['CG_A_02'].rolling(window=n_per, min_periods=n_per).std().reset_index(0,drop=True)
df['DesvPad_CG_H_02'] = df.groupby('Home')['DesvPad_CG_H_02'].shift(1)
df['DesvPad_CG_A_02'] = df.groupby('Away')['DesvPad_CG_A_02'].shift(1)
df['CV_CG_H_02'] = df['DesvPad_CG_H_02'] / df['Media_CG_H_02']
df['CV_CG_A_02'] = df['DesvPad_CG_A_02'] / df['Media_CG_A_02']

colunas_deletar = ['Ptos_H','Ptos_A','DesvPad_Ptos_H','DesvPad_Ptos_A',
                   'DesvPad_GM_H','DesvPad_GM_A',
                   'DesvPad_GS_H','DesvPad_GS_A',
                   'SG_H','SG_A','DesvPad_SG_H','DesvPad_SG_A',
                   'SG_H_01','SG_A_01','DesvPad_SG_H_01','DesvPad_SG_A_01',
                   'SG_H_02','SG_A_02','DesvPad_SG_H_02','DesvPad_SG_A_02',
                   'VG_H','VG_A','DesvPad_VG_H','DesvPad_VG_A',
                   'CG_H_00','CG_A_00','DesvPad_CG_H_00','DesvPad_CG_A_00',
                   'CG_H_01','CG_A_01','DesvPad_CG_H_01','DesvPad_CG_A_01',
                   'CG_H_02','CG_A_02','DesvPad_CG_H_02','DesvPad_CG_A_02']

df = df.drop(colunas_deletar, axis=1)
df = drop_reset_index(df)

display(df)

In [ ]:
df = df[df['Season'].isin(['2021/2022', '2022/2023', '2023/2024','2024/2025']) == True]
df = drop_reset_index(df)
display(df)

### Método de Lay Away

In [ ]:
flt = ((df.CV_VG_H >= 1.6678) & (df.CV_VG_H <= 1.7848)) | ((df.CV_CG_H_01 >= 1.7026) & (df.CV_CG_H_01 <= 1.8094))
df_filtrado = df[flt].copy()
df_filtrado = drop_reset_index(df_filtrado)
display(df_filtrado)

In [ ]:
df_filtrado['Profit'] = np.where((df_filtrado['Goals_H_FT'] >= df_filtrado['Goals_A_FT']),
                                 df_filtrado['Odd_1X']-1, -1)
display(df_filtrado)

In [ ]:
df_filtrado['Profit_Acu'] = df_filtrado['Profit'].cumsum()
df_filtrado['Profit_Acu'].plot()